# __[Study] 4. JSON, XML 등 YOLO와 다른 라벨 포맷 데이터 파싱하기__
- Parsing?<br>
JSON, XML 등에서 내가 원하는 데이터를 특정 패턴이나 순서로 추출하여 정보로 가공하는 것을 말합니다.<br>
'공공 데이터 포탈', 'AI HUB', '서울시 열린데이터' 등 제공하는 데이터들이 보통 .JSON 파일 포맷으로 이미지가 라벨링되어 있으며, <br>
라벨링 된 패턴이 데이터마다 상이하여 JSON 파일을 탐색하고 <br>
YOLOv5 라벨 데이터 .txt(class_id center_x center_y Dw Dh) 형태로 Convert하는 작업이 필요함. 

- 실습 파일 4번은 AI HUB 내 <font color="red">공사현장 안전장비 인식 이미지"</font> 데이터의 일부를 사용할 예정입니다. <br>
https://www.aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=163

---

## 0. 환경 설정하기

### 1) 구글 드라이브 연결하기

In [1]:
# 코랩 사용 시 구글 드라이브 연결
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install xmltodict

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### 2) 경로 확인하기
- "WORK_SPACE" 에 본인 작업 경로 작성 후 실행(구글 드라이브 최상위에 압축해제 시 그대로 실행. 수정 X).<br>

<font color="red">※ 주의. 나머지 경로는 절대 변경하지 마세요.</font>

### 3) YOLOv5 파일 다운로드 및 설치
![install](https://github.com/DrKAI/CV/raw/main/UltraLytics_manual/yolov5_install.png)

[Install Page](https://github.com/ultralytics/yolov5)

In [ ]:
# UltraLytics git에서 복사하기
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -r requirements.txt

### 4) 라이브러리 불러오기
필요시 추가 라이브러리는 설치해서 사용하세요.

In [5]:
# 필요 라이브러리 불러오기.
import glob
import yaml
import json
import pandas as pd
from PIL import Image
from IPython.display import Image
import zipfile
import gdown
import os
import xmltodict
import xml.etree.ElementTree as ET
from tqdm.auto import tqdm, trange

## 데이터 증강 Salt & Pepper 노이즈

In [ ]:
data['xmax'][0]

419

In [6]:
import cv2
from google.colab.patches import cv2_imshow
import copy
from random import randint
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
def SaltPepper(img):
    # Getting the dimensions of the image
    if img.ndim > 2:  # color
        height, width, _ = img.shape
    else:  # gray scale
        height, width = img.shape
    result = copy.deepcopy(img)

    # Randomly pick some pixels in the image
    # Pick a random number between height*width/80 and height*width/10
    number_of_pixels = randint(int(height * width / 100), int(height * width / 10))

    for i in range(number_of_pixels):
        # Pick a random y coordinate
        y_coord = randint(0, height - 1)

        # Pick a random x coordinate
        x_coord = randint(0, width - 1)

        if result.ndim > 2:
            result[y_coord][x_coord] = [0, 0, 0]
        else:
            # Color that pixel to white
            result[y_coord][x_coord] = 255

    # Randomly pick some pixels in image
    # Pick a random number between height*width/80 and height*width/10
    for i in range(number_of_pixels):
        # Pick a random y coordinate
        y_coord = randint(0, height - 1)

        # Pick a random x coordinate
        x_coord = randint(0, width - 1)

        if result.ndim > 2:
            result[y_coord][x_coord] = [0,0,0]
        else:
            # Color that pixel to white
            result[y_coord][x_coord] = 0

    return result
'''
img=cv2.imread('/content/drive/MyDrive/kt_aivle/car_plate/data/train/images/Cars0.png')
result = SaltPepper(img)
cv2_imshow(result)
cv2.imwrite('/content/drive/MyDrive/kt_aivle/car_plate/data/train/image_pepper/Cars0_pepper.png',img)
'''
for i in range(433):
  print(i)
  img=cv2.imread(f'/content/drive/MyDrive/kt_aivle/car_plate/data/train/images/Cars{i}.png')
  result = SaltPepper(img)
  #cv2_imshow(result)
  cv2.imwrite(f'/content/drive/MyDrive/kt_aivle/car_plate/data/train/image_pepper/Cars{i}_pepper.png',result)

---

In [8]:
data=pd.read_csv('/content/drive/MyDrive/kt_aivle/car_plate/annotation_dataframe.csv')

In [4]:
data['file']=data['file'].str.replace('.','_pepper.')

NameError: ignored

In [ ]:
data.to_csv('annotation_pepper_dataframe.csv')

In [15]:
data

,file,width,height,xmin,ymin,xmax,ymax
0,Cars0.png,500,268,226,125,419,173
1,Cars1.png,400,248,134,128,262,160
2,Cars10.png,400,225,140,5,303,148
3,Cars100.png,400,267,175,114,214,131
4,Cars101.png,400,300,167,202,240,220
...,...,...,...,...,...,...,...
466,Cars95.png,600,400,278,182,407,263
467,Cars96.png,400,248,133,126,261,160
468,Cars97.png,400,300,98,102,297,191
469,Cars98.png,370,400,85,196,247,259


In [ ]:
%cd ..

/content/drive/MyDrive/kt aivle/car_plate/data/train


In [ ]:
%cd /content/drive/MyDrive/kt aivle/car_plate/data/train/train_image
!unzip -qq '/content/drive/MyDrive/kt_aivle/car_plate/data/train/train_image/train_image.zip' # 압축파일 경로

/content/drive/MyDrive/kt aivle/car_plate/data/train/train_image


In [9]:
annotation_dataframe=pd.read_csv('/content/drive/MyDrive/kt_aivle/car_plate/annotation_dataframe.csv')
annotation_pepper_dataframe=pd.read_csv('/content/drive/MyDrive/kt_aivle/car_plate/annotation_pepper_dataframe.csv',index_col=0)

In [ ]:
annotation_pepper_dataframe['class']=1
annotation_pepper_dataframe['Xcent']=(annotation_pepper_dataframe['xmin']+annotation_pepper_dataframe['xmax'])/2/annotation_pepper_dataframe['width']
annotation_pepper_dataframe['Ycent']=(annotation_pepper_dataframe['ymin']+annotation_pepper_dataframe['ymax'])/2/annotation_pepper_dataframe['height']
annotation_pepper_dataframe['boxW']=(annotation_pepper_dataframe['xmax']-annotation_pepper_dataframe['xmin'])/annotation_pepper_dataframe['width']
annotation_pepper_dataframe['boxH']=(annotation_pepper_dataframe['ymax']-annotation_pepper_dataframe['ymin'])/annotation_pepper_dataframe['height']
annotation_pepper_dataframe['name'] = 'license'

In [13]:
df.to_csv('df_pepper_dataframe.csv')

In [ ]:
%cd /content

/content


In [10]:
df = pd.concat([annotation_dataframe,annotation_pepper_dataframe],axis=0)
df=df.reset_index(drop=True)

In [11]:
df['class']=1
df['Xcent']=(df['xmin']+df['xmax'])/2/df['width']
df['Ycent']=(df['ymin']+df['ymax'])/2/df['height']
df['boxW']=(df['xmax']-df['xmin'])/df['width']
df['boxH']=(df['ymax']-df['ymin'])/df['height']
df['name'] = 'license'

In [12]:
df

,file,width,height,xmin,ymin,xmax,ymax,class,Xcent,Ycent,boxW,boxH,name
0,Cars0.png,500,268,226,125,419,173,1,0.645000,0.555970,0.386000,0.179104,license
1,Cars1.png,400,248,134,128,262,160,1,0.495000,0.580645,0.320000,0.129032,license
2,Cars10.png,400,225,140,5,303,148,1,0.553750,0.340000,0.407500,0.635556,license
3,Cars100.png,400,267,175,114,214,131,1,0.486250,0.458801,0.097500,0.063670,license
4,Cars101.png,400,300,167,202,240,220,1,0.508750,0.703333,0.182500,0.060000,license
...,...,...,...,...,...,...,...,...,...,...,...,...,...
937,Cars95_pepper.png,600,400,278,182,407,263,1,0.570833,0.556250,0.215000,0.202500,license
938,Cars96_pepper.png,400,248,133,126,261,160,1,0.492500,0.576613,0.320000,0.137097,license
939,Cars97_pepper.png,400,300,98,102,297,191,1,0.493750,0.488333,0.497500,0.296667,license
940,Cars98_pepper.png,370,400,85,196,247,259,1,0.448649,0.568750,0.437838,0.157500,license


In [ ]:
imgs = os.listdir('/content/drive/MyDrive/kt_aivle/car_plate/data/train/train_image')
train_imgs, valid_imgs = train_test_split(imgs, test_size=0.05, random_state=2022)

In [ ]:
import yaml
cwd = '/content/drive/MyDrive/kt_aivle/car_plate/data/train/'
data = dict(
    train = cwd + 'train.txt',
    val = cwd + 'val.txt',
    nc = 1,
    names = ['license'],
)

with open(cwd + 'bgr.yaml', 'w') as outfile:
  yaml.dump(data, outfile, default_flow_style=False)
with open(cwd + 'train.txt', 'w') as f:
  for path in train_imgs:
    f.write(cwd+'train_image/'+path+'\n')
with open(cwd + 'val.txt', 'w') as f:
  for path in valid_imgs:
    f.write(cwd+'train_image/'+path+'\n')

In [146]:
for file in imgs:
  #file = file.split('.')[0]
  bboxs = []
  for _, row in df[df['file'] == file].iterrows():
    bbox = [str(0), str(row['Xcent']), str(row['Ycent']), str(row['boxW']), str(row['boxH'])]
    bbox = ' '.join(bbox)
    bboxs.append(bbox)
  #file = file.split('.')[0]
  with open(cwd+'labels/'+file+'.txt', 'w') as f:
    bboxs = '\n'.join(bboxs)
    f.write(bboxs)

In [ ]:
%cd yolov5
%pip install -qr requirements.txt

/content/yolov5


In [ ]:
%cd yolov5

/content/yolov5


In [147]:
!python train.py --img 640\
 --batch 16\
  --epochs 30\
   --data /content/drive/MyDrive/kt_aivle/car_plate/data/train/bgr.yaml\
    --weights yolov5s.pt --name car_license_detection

train: weights=yolov5s.pt, cfg=, data=/content/drive/MyDrive/kt_aivle/car_plate/data/train/bgr.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=30, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=car_license_detection, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.2-206-gfba61e5 Python-3.7.15 torch-1.12.1+cu113 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=